# Простейшая модель инвертированного индекса

Блокнот содержит:
- тестирование реализованного инвертированного индекса, реализованных алгоритмов сжатия
- использование индекса на данных, полученных в модуле 2
- использование индекса с алгоритмами сжатия
- сравнение эффективности

In [2]:
from index import InvertedIndex
from coding import EliasCoder

In [9]:
from pympler import asizeof

## Тестирование реализации индекса
- токенизация ✔
- лемматизация ✔
- индексирование ✔
- поиск ✔

In [17]:
index_test = InvertedIndex()

In [4]:
sentence = 'Математика — это наука о числах, формах и закономерностях.'
print(sentence)

tokenized_sentence = index_test._tokenize(text=sentence)
print('После разбиения на токены:', tokenized_sentence)

lemmatized_sentence = index_test._lemmatize(tokens=tokenized_sentence)
print('После разбиения на токены:', lemmatized_sentence)

Математика — это наука о числах, формах и закономерностях.
После разбиения на токены: ['математика', 'это', 'наука', 'о', 'числах', 'формах', 'и', 'закономерностях']
После разбиения на токены: ['математик', 'это', 'наука', 'число', 'форма', 'закономерность']


In [18]:
# индексирование
# 10 предложений для тестирования сгенерированы с помощью LLM
index_test.add_document(1, "Математика — это наука о числах, формах и закономерностях.")
index_test.add_document(2, "Пифагорова теорема описывает связь между сторонами прямоугольного треугольника.")
index_test.add_document(3, "Алгебра изучает операции и отношения между числами.")
index_test.add_document(4, "Геометрия исследует свойства фигур и пространственные отношения.")
index_test.add_document(5, "Калькулятор помогает выполнять сложные математические вычисления.")
index_test.add_document(6, "Интегралы и производные являются основными понятиями в математическом анализе.")
index_test.add_document(7, "Статистика позволяет анализировать и интерпретировать данные.")
index_test.add_document(8, "Теория вероятностей изучает случайные события и их закономерности.")
index_test.add_document(9, "Математическая логика формулирует правила и принципы рассуждения.")
index_test.add_document(10, "Функции связывают элементы одного множества с элементами другого множества.")

index_test

In [4]:
# поиск
query = "изучает"
results = index_test.search(query)
print(f"Документы, содержащие {query}: {results}")

# извлечение документа
for doc_id in results:
    print(f"Документ {doc_id}:\n{index_test.get_document(doc_id)}\n")

Документы, содержащие изучает: [8, 3]
Документ 8:
Теория вероятностей изучает случайные события и их закономерности.

Документ 3:
Алгебра изучает операции и отношения между числами.



## Тестирование реализации гамма/дельта кодирования/декодирования Элиаса

- гамма-кодирование ✔
- гамма-декодирование ✔
- дельта-кодирование ✔
- дельта-декодирование ✔

In [2]:
coder = EliasCoder()

In [3]:
# гамма-кодирование
for number in [15, 19, 25, 27]: # 0001111 000010011 000011001 000011011
    gamma_number = coder.gamma_encoding(number)
    print(f'Число {number}, гамма-представление: {gamma_number}')

Число 15, гамма-представление: 0001111
Число 19, гамма-представление: 000010011
Число 25, гамма-представление: 000011001
Число 27, гамма-представление: 000011011


In [5]:
# гамма-декодирование
for number in [109828, 20929, 2010, 1]: 
    gamma_decoded_number = coder.gamma_decoding(coder.gamma_encoding(number))
    print(f'Число {number}, гамма-декодированное число: {gamma_decoded_number}')

Число 109828, гамма-декодированное число: 109828
Число 20929, гамма-декодированное число: 20929
Число 2010, гамма-декодированное число: 2010
Число 1, гамма-декодированное число: 1


In [7]:
# дельта-кодирование
for number in [7, 8, 9]: # 01111 00100000 00100001
    delta_number = coder.delta_encoding(number)
    print(f'Число {number}, дельта-представление: {delta_number}')

Число 7, дельта-представление: 01111
Число 8, дельта-представление: 00100000
Число 9, дельта-представление: 00100001


In [6]:
# дельта-декодирование
for number in [344431, 871, 324, 141]: 
    delta_decoded_number = coder.delta_decoding(coder.delta_encoding(number))
    print(f'Число {number}, дельта-декодированное число: {delta_decoded_number}')

Число 344431, дельта-декодированное число: 344431
Число 871, дельта-декодированное число: 871
Число 324, дельта-декодированное число: 324
Число 141, дельта-декодированное число: 141


## Тестирование сжатия индекса
- после сжатия ссылки на документы стали закодированными
- размер индекса уменьшился

In [19]:
# индекс до сжатия
index_test_orig = index_test.get_index()

print('До сжатия размер в КБ:', asizeof.asizeof(index_test_orig) / 1024)
for i, key in enumerate(index_test_orig.keys()):
    if i == 3: break
    print(key, ':', index_test_orig[key])

До сжатия размер в КБ: 11.7734375
математик : [1]
это : [1]
наука : [1]


In [20]:
index_test.compress_index(method='gamma')

# индекс после сжатия
index_test_gamma = index_test.get_index()
print('После gamma-сжатия размер в КБ:', asizeof.asizeof(index_test_gamma) / 1024)
for i, key in enumerate(index_test_gamma.keys()):
    if i == 3: break
    print(key, ':', index_test_gamma[key])

После gamma-сжатия размер в КБ: 11.4609375
математик : bitarray('1')
это : bitarray('1')
наука : bitarray('1')


In [6]:
from bitarray import bitarray

coder = EliasCoder()
b = bitarray(coder.gamma_encoding(15))
b

bitarray('0001111')

In [10]:
b.to01()

'0001111'